## Homework 5
### Minjie Fan 998585352

### Exercise 4

In [2]:
using ReverseDiffSource

#### Copy Newton and Quasi-Newton from the previous homework

In [67]:
function backtrack( obj, grd, x, d, rho=0.9, alpha=1.0, c=1e-4 )
    # Find step size by backtracking
    gxp = (grd(x)'*d)[1]
    while obj(x+alpha*d)>obj(x)+c*alpha*gxp
        alpha *= rho
    end
    return alpha
end

backtrack (generic function with 4 methods)

In [68]:
# Use the template provided
function newtmin( obj, grd, hes, x, maxIts=100, optTol=1e-6, modify=1 )
    # the sqrt of the machine precision
    delta = 1e-8
    n = length(x)
    t = NaN
    d = NaN
    for t = 1:maxIts
        f = obj(x)
        g = grd(x)
        H = hes(x)
        # check condition for breaking
        if norm(g)<optTol
            break
        end
        # try chol decomp first
        try
            F = chol(H)
            d = -R\(R'\g)
        # o.w., use eigenvalue modification
        catch
            F = eigfact(H)
            # method 1
            if modify==1
                Lambda_inv = 1./max(abs(F[:values]), delta)
            # method 2
            else
                Lambda_inv = 1./max(F[:values], delta)
            end
            d = -F[:vectors]*Diagonal(Lambda_inv)*F[:vectors]'*g
        end
        # find alpha
        alpha = backtrack( obj, grd, x, d )
        # update x
        x = x+alpha*d
    end
    return (x, t-1)
end

newtmin (generic function with 4 methods)

In [177]:
function bisection( obj, grd, x, d, rho=0.9, alpha=1.0, alpha_l=0.0, alpha_r=100.0, c1=1e-4, c2=0.9, max_iter=100 )
    # Find step size by bisection method
    for i = 1:max_iter
        gxp = (grd(x)'*d)[1]
        x_new = x+alpha*d
        gxp_new = (grd(x_new)'*d)[1]
        if obj(x_new)>obj(x)+c1*alpha*gxp
            alpha_r = alpha
            alpha = (alpha_l+alpha_r)/2.0
        elseif gxp_new<c2*gxp
            alpha_l = alpha
            if alpha_r>=100.0
                alpha = 2*alpha_l
            else
                alpha = (alpha_l+alpha_r)/2.0
            end
        else 
            break
        end
    end
    return alpha
end

bisection (generic function with 8 methods)

In [178]:
# Use the template provided
function newtminBFGS( obj, grd, hes, x, maxIts=100, optTol=1e-6 )
    # Minimize a function f using Newton’s method.
    n = length(x)
    # the init value of H
    H_approx = eye(n)
    t = NaN
    f_pre = NaN
    for t = 1:maxIts
        f = obj(x)
        g = grd(x)
        H = hes(x)
        if norm(g)<optTol
            break
        end
        # get dir
        d = -H_approx*g
        # find alpha
        alpha = bisection( obj, grd, x, d )
        # update x
        x_new = x+alpha*d
        # get s
        s = x_new-x
        # get y
        y = grd(x_new)-g
        rho_k = 1/(y'*s)[1]
        # update H
        H_approx = (eye(n)-rho_k*s*y')*H_approx*(eye(n)-rho_k*y*s')+rho_k*s*s'
        
        # update x and f_pre
        x = x_new
        f_pre = f
    end
    return (x, t-1)
end

newtminBFGS (generic function with 3 methods)

#### Define the augmented Lagrangian

In [146]:
function L(x, y, rho, f, c)
    c_vec = c(x)
    m = length(c_vec)
    value = f(x)
    for i = 1:m
        value = value-y[i]*c_vec[i]+0.5*rho*c_vec[i]^2
    end
    return value
end

L (generic function with 1 method)

#### Define the gradient of L

In [147]:
function gL(x, y, rho, c, f_diff, c_diff)
    n = length(x)
    m = length(y)
    vec = zeros(n, 1)
    g_vec = f_diff(x)[2]
    J_mat = zeros(m, n)
    for j = 1:m
        J_mat[j, :] = c_diff[j](x)[2] 
    end
    y_hat = y-rho*c(x)
    for i = 1:n
        tmp = 0
        for j = 1:m
            tmp = tmp+J_mat[j, i]*y_hat[j]
        end
        vec[i] = g_vec[i]-tmp
    end
    return vec
end

gL (generic function with 2 methods)

#### Define the Hessian of L

In [150]:
function HL(x, y, rho, c, f_diff, c_diff)
    n = length(x)
    m = length(y)
    y_hat = y-rho*c(x)
    mat = f_diff(x)[3]
    for j = 1:m
        mat = mat-y_hat[j]*c_diff[j](x)[3]
    end
    J_mat = zeros(m, n)
    for j = 1:m
        J_mat[j, :] = c_diff[j](x)[2] 
    end
    mat = mat+rho*(J_mat'*J_mat)
end

HL (generic function with 2 methods)

#### Test L, gL and HL

In [154]:
f(x) = (1-x[1])^2 

f (generic function with 1 method)

In [155]:
c(x) = 10*(x[2]-x[1]^2)

c (generic function with 1 method)

In [156]:
f_diff = rdiff(f, (ones(2),), order=2)

(anonymous function)

In [157]:
c_diff = rdiff(c, (ones(2),), order=2)

(anonymous function)

In [158]:
L([1, 1], 1, 1, f, c)

0.0

In [159]:
gL([1, 1], 1, 1, c, f_diff, (c_diff,))

2x1 Array{Float64,2}:
  20.0
 -10.0

In [160]:
HL([1, 1], 1, 1, c, f_diff, (c_diff,))

2x2 Array{Float64,2}:
  422.0  -200.0
 -200.0   100.0

#### Define the function of the augmented Lagrangian method

In [179]:
function AL(f, c, f_diff, c_diff, x, y, rho, optFun, maxIts=100, optTol=1e-6)
    # note that c_diff is a tuple of functions
    t = NaN
    for t = 1:maxIts
        obj(x) = L(x, y, rho, f, c)
        grd(x) = gL(x, y, rho, c, f_diff, c_diff)
        hes(x) = HL(x, y, rho, c, f_diff, c_diff)
        # check whether exit
        if norm(grd(x))<optTol
            break
        end
        # approximate minimization is enough
        if optFun=="Newton"
            res = newtmin( obj, grd, hes, x, 100, 1e-6 )
        elseif optFun=="BFGS"
            res = newtminBFGS( obj, grd, hes, x, 100, 1e-6 )
        end
        x = res[1]
        # check whether ||c(x)|| is small enough
        if norm(c(x))<1e-3
            y = y-rho*c(x)
        else
            # increase rho by a factor of 2
            rho = rho*2
        end
    end
    return (x, t-1)
end

AL (generic function with 4 methods)

#### Problem 6

In [180]:
f(x) = (1-x[1])^2 

f (generic function with 1 method)

In [181]:
c(x) = 10*(x[2]-x[1]^2)

c (generic function with 1 method)

In [182]:
f_diff = rdiff(f, (ones(2),), order=2)

(anonymous function)

In [183]:
c_diff = rdiff(c, (ones(2),), order=2)

(anonymous function)

In [194]:
tic()
res = AL(f, c, f_diff, (c_diff,), [-1.2, 1], 1, 1, "Newton", 1e3)
time = toc();

elapsed time: 0.026284256 seconds


In [195]:
res

(
2x1 Array{Float64,2}:
 1.0
 1.0,

12.0)

In [196]:
tic()
res = AL(f, c, f_diff, (c_diff,), [-1.2, 1], 1, 1, "BFGS", 1e3)
toc();

elapsed time: 0.02304561 seconds


In [197]:
res

(
2x1 Array{Float64,2}:
 1.0
 1.0,

12.0)

For problem 6, both Newton and quasi-Newton implementations give the correct minimum. The former is a little bit slower than the latter (0.026284256 seconds>0.02304561 seconds).

#### Problem 7

In [198]:
f(x) = log(1+x[1]^2)-x[2]

f (generic function with 1 method)

In [199]:
c(x) = (1+x[1]^2)^2+x[2]^2-4

c (generic function with 1 method)

In [200]:
f_diff = rdiff(f, (ones(2),), order=2)

(anonymous function)

In [201]:
c_diff = rdiff(c, (ones(2),), order=2)

(anonymous function)

In [237]:
tic()
res = AL(f, c, f_diff, (c_diff,), [2, 2], 1, 1, "Newton", 1e3)
toc();

elapsed time: 0.012054674 seconds


In [226]:
res

(
2x1 Array{Float64,2}:
 8.15206e-65
 1.73205    ,

14.0)

In [227]:
norm(res[1]-[0, sqrt(3)])

9.969802761133906e-14

In [238]:
tic()
res = AL(f, c, f_diff, (c_diff,), [2, 2], 1, 1, "BFGS", 1e3)
toc();

elapsed time: 0.015254914 seconds


In [229]:
res

(
2x1 Array{Float64,2}:
 4.18172e-10
 1.73205    ,

14.0)

In [230]:
norm(res[1]-[0, sqrt(3)])

4.1817352094597014e-10

For problem 7, both Newton and quasi-Newton implementations give the correct minimum with tiny numerical error. The former has smaller error and also runs faster than the latter.

#### Problem 8

In [239]:
f(x) = -1

f (generic function with 1 method)

In [240]:
c1(x) = x[1]^2+x[2]^2-25

c1 (generic function with 1 method)

In [241]:
c2(x) = x[1]*x[2]-9

c2 (generic function with 1 method)

In [242]:
c(x) = [c1(x), c2(x)]

c (generic function with 1 method)

In [243]:
f_diff_const(x) = (-1, zeros(2), zeros(2,2))

f_diff_const (generic function with 1 method)

In [244]:
c1_diff = rdiff(c1, (ones(2),), order=2)

(anonymous function)

In [245]:
c2_diff = rdiff(c2, (ones(2),), order=2)

(anonymous function)

In [260]:
tic()
res = AL(f, c, f_diff_const, (c1_diff, c2_diff,), [2, 1], [1, 1], 1, "Newton", 1e3)
toc();

elapsed time: 0.013045841 seconds


In [261]:
res

(
2x1 Array{Float64,2}:
 4.60159
 1.95584,

13.0)

In [262]:
tic()
res = AL(f, c, f_diff_const, (c1_diff, c2_diff,), [2, 1], [1, 1], 1, "BFGS", 1e3)
toc();

elapsed time: 0.028363099 seconds


In [263]:
res

(
2x1 Array{Float64,2}:
 4.60159
 1.95584,

13.0)

For problem 8, both Newton and quasi-Newton implementations give one of the correct minimums. The former runs much faster than the latter.

#### Problem 9

In [264]:
f(x) = sin(pi*x[1]/12)*cos(pi*x[2]/16)

f (generic function with 1 method)

In [265]:
c(x) = 4*x[1]-3*x[2]

c (generic function with 1 method)

In [266]:
f_diff = rdiff(f, (ones(2),), order=2)

(anonymous function)

In [267]:
c_diff = rdiff(c, (ones(2),), order=2)

(anonymous function)

In [274]:
tic()
res = AL(f, c, f_diff, (c_diff,), [0, 0], 1, 1, "Newton", 1e3)
toc();

elapsed time: 0.00599479 seconds


In [275]:
res

(
2x1 Array{Float64,2}:
 -15243.0
 -20324.0,

12.0)

In [276]:
tic()
res = AL(f, c, f_diff, (c_diff,), [0, 0], 1, 1, "BFGS", 1e3)
toc();

elapsed time: 0.011753298 seconds


In [277]:
res

(
2x1 Array{Float64,2}:
 -3.0
 -4.0,

12.0)

For problem 9, both Newton and quasi-Newton implementations give one of the correct minimums. The most interesting finding is that the minimums they give are different. Besides, the former runs much faster than the latter.